In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.regularizers import L1
from tensorflow.keras import backend as K
import keras
from keras import ops
from keras import layers

2025-04-01 11:33:06.122664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-01 11:33:07.731050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
student_trn = pd.read_csv('student_trn.csv').iloc[:, 1:]
student_trn = student_trn[student_trn['y'] == 1].drop('y', axis = 1)
student_trn, student_val = train_test_split(student_trn, test_size = 0.1, random_state = 200)
student_trn = student_trn.reset_index().drop('index', axis=1)
student_val = student_val.reset_index().drop('index', axis=1)
student_tst = pd.read_csv('student_tst.csv').iloc[:, 1:]

student_enc = OneHotEncoder(sparse_output=False)
enc_cols = student_trn.select_dtypes('object')

encoded = pd.DataFrame(student_enc.fit_transform(enc_cols), 
                      columns = student_enc.get_feature_names_out())
student_trn.drop(enc_cols.columns, axis = 1, inplace=True)
student_trn = pd.concat([student_trn, encoded], axis = 1)

encoded = pd.DataFrame(student_enc.transform(student_tst.select_dtypes('object')),
                         columns = student_enc.get_feature_names_out())
student_tst.drop(enc_cols.columns, axis = 1, inplace=True)
student_tst = pd.concat([student_tst, encoded], axis = 1)

encoded = pd.DataFrame(student_enc.transform(student_val.select_dtypes('object')),
                         columns = student_enc.get_feature_names_out())
student_val.drop(enc_cols.columns, axis = 1, inplace=True)
student_val = pd.concat([student_val, encoded], axis = 1)
student_cols = student_trn.columns

student_scaler = MinMaxScaler()
student_trn = student_scaler.fit_transform(student_trn)
student_tst = student_scaler.transform(student_tst)
student_val = student_scaler.transform(student_val)


In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator()

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

class VEncoder(keras.Model):
    def __init__(self, input_dim: int, latent_dim:int, **kwargs):
        super().__init__(**kwargs)

        encoder_inputs = keras.Input(shape=(input_dim,))
        x = layers.Dense(512, activation = "relu")(encoder_inputs)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(128, activation = "relu")(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(32, activation = "relu")(x)
        z_mean = layers.Dense(latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
        z = Sampling()([z_mean, z_log_var])
        self.encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    def call(self, x):
        return self.encoder(x)

class VDecoder(keras.Model):
    def __init__(self, latent_dim:int, output_dim:int, **kwargs):
        super().__init__(**kwargs)

        latent_inputs = keras.Input(shape=(latent_dim,))
        x = layers.Dense(32, activation = "relu")(latent_inputs)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(128, activation = "relu")(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(512, activation = "relu")(x)
        x = layers.Dropout(0.25)(x)
        decoder_outputs = layers.Dense(output_dim, activation="sigmoid")(x)
        
        self.decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    def call(self, x):
        return self.decoder(x)


class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
    
    def call(self, x):
        z_mean, z_log_var, z = self.encoder(x)
        reconstruction = self.decoder(z)
        return z_mean, z_log_var, reconstruction

    # also used during validation
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def calculate_reconstruction_loss(self, data, reconstruction):
        """
        In case of computer vision tasks use the following:
            keras.losses.binary_crossentropy(data, reconstruction),
            axis=(1, 2),
        """
        return ops.mean(ops.sum(keras.losses.binary_crossentropy(data, reconstruction)))
    
    def calculate_kl_loss(self, z_mean, z_log_var):
        kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
        kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
        return kl_loss
    
    def calculate_total_loss(self, reconstruction_loss, kl_loss):
        return reconstruction_loss + kl_loss 
        
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            
            reconstruction_loss = self.calculate_reconstruction_loss(data, reconstruction)
            kl_loss = self.calculate_kl_loss(z_mean, z_log_var)
            total_loss = self.calculate_total_loss(reconstruction_loss, kl_loss)

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }
    
    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data, training=False)
        reconstruction = self.decoder(z)
        
        reconstruction_loss = self.calculate_reconstruction_loss(data, reconstruction)
        kl_loss = self.calculate_kl_loss(z_mean, z_log_var)
        total_loss = self.calculate_total_loss(reconstruction_loss, kl_loss)
        
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [4]:
input_dim = student_trn.shape[1] 

## DZ = 1

In [5]:
vaes = []
for i in range(1, 34):
    encoder = VEncoder(input_dim, i)
    decoder = VDecoder(i, input_dim)
    
    vaes.append(VAE(encoder, decoder))
    vaes[i - 1].compile(optimizer='adam')

2025-04-01 11:33:24.997070: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-01 11:33:24.997440: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
vaes

[<VAE name=vae, built=False>,
 <VAE name=vae_1, built=False>,
 <VAE name=vae_2, built=False>,
 <VAE name=vae_3, built=False>,
 <VAE name=vae_4, built=False>,
 <VAE name=vae_5, built=False>,
 <VAE name=vae_6, built=False>,
 <VAE name=vae_7, built=False>,
 <VAE name=vae_8, built=False>,
 <VAE name=vae_9, built=False>,
 <VAE name=vae_10, built=False>,
 <VAE name=vae_11, built=False>,
 <VAE name=vae_12, built=False>,
 <VAE name=vae_13, built=False>,
 <VAE name=vae_14, built=False>,
 <VAE name=vae_15, built=False>,
 <VAE name=vae_16, built=False>,
 <VAE name=vae_17, built=False>,
 <VAE name=vae_18, built=False>,
 <VAE name=vae_19, built=False>,
 <VAE name=vae_20, built=False>,
 <VAE name=vae_21, built=False>,
 <VAE name=vae_22, built=False>,
 <VAE name=vae_23, built=False>,
 <VAE name=vae_24, built=False>,
 <VAE name=vae_25, built=False>,
 <VAE name=vae_26, built=False>,
 <VAE name=vae_27, built=False>,
 <VAE name=vae_28, built=False>,
 <VAE name=vae_29, built=False>,
 <VAE name=vae_30, bui

In [7]:
for i in range(1, 34):
    history = vaes[i-1].fit(student_trn,
                          epochs = 100,
                          batch_size = 128, 
                          validation_data = student_val,
                          shuffle = True
                         )
    student_vae = vaes[i-1].predict(student_tst)[2]
    student_vae = pd.DataFrame(student_scaler.inverse_transform(student_vae), 
                             columns = student_cols)
    enc_cols = student_enc.inverse_transform(student_vae[student_enc.get_feature_names_out()])
    student_vae = student_vae.drop(student_enc.get_feature_names_out(), axis = 1)
    student_vae[student_enc.feature_names_in_] = enc_cols
    path = "dz_vae/student" + str(i) + ".csv" 
    student_vae.to_csv(path, index=False)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 197ms/step - kl_loss: 0.0173 - loss: 85.0917 - reconstruction_loss: 85.0744 - val_kl_loss: 0.0379 - val_loss: 35.0514 - val_reconstruction_loss: 35.0134
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - kl_loss: 0.1040 - loss: 82.1014 - reconstruction_loss: 81.9974 - val_kl_loss: 0.5971 - val_loss: 33.0150 - val_reconstruction_loss: 32.4179
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - kl_loss: 1.1061 - loss: 76.9713 - reconstruction_loss: 75.8653 - val_kl_loss: 2.3494 - val_loss: 32.9503 - val_reconstruction_loss: 30.6009
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - kl_loss: 1.9557 - loss: 73.2623 - reconstruction_loss: 71.3066 - val_kl_loss: 0.7794 - val_loss: 30.4911 - val_reconstruction_loss: 29.7116
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - kl_loss: 0.6508 - loss: 70.4046 - reconstruction_loss: 69.7538 - val_kl_loss: 0.2851 - val_loss: 29.2457 - val_reconstruction_loss: 28.9606
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - kl_loss: 1.2039 - loss: 61.2226 - reconstruction_loss: 60.0187 - val_kl_loss: 1.2040 - val_loss: 26.7096 - val_reconstruction_loss: 25.5056
Epoch 38/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - kl_loss: 1.2268 - loss: 60.8717 - reconstruction_loss: 59.6448 - val_kl_loss: 1.2348 - val_loss: 26.6221 - val_reconstruction_loss: 25.3873
Epoch 39/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - kl_loss: 1.2467 - loss: 60.7931 - reconstruction_loss: 59.5464 - val_kl_loss: 1.2373 - val_loss: 26.6410 - val_reconstruction_loss: 25.4037
Epoch 40/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - kl_loss: 1.2543 - loss: 61.1132 - reconstruction_loss: 59.8589 - val_kl_loss: 1.3034 - val_loss: 26.6940 - val_reconstruction_loss: 25.3906
Epoch 41/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - kl_loss: 1.3027 - loss: 61.0524 - reconstruction_loss: 59.7497 - val_kl_loss: 1.2816 - val_loss: 26.7216 - val_reconstruction_loss: 25.4400
Epoch 42/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/st

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - kl_loss: 1.5056 - loss: 59.9674 - reconstruction_loss: 58.4618 - val_kl_loss: 1.4650 - val_loss: 26.7240 - val_reconstruction_loss: 25.2590
Epoch 74/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - kl_loss: 1.5325 - loss: 60.1148 - reconstruction_loss: 58.5823 - val_kl_loss: 1.4152 - val_loss: 26.7648 - val_reconstruction_loss: 25.3497
Epoch 75/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - kl_loss: 1.3984 - loss: 60.5858 - reconstruction_loss: 59.1874 - val_kl_loss: 1.4477 - val_loss: 26.6691 - val_reconstruction_loss: 25.2215
Epoch 76/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - kl_loss: 1.4957 - loss: 60.1251 - reconstruction_loss: 58.6294 - val_kl_loss: 1.4643 - val_loss: 26.7421 - val_reconstruction_loss: 25.2778
Epoch 77/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - kl_loss: 1.5789 - loss: 60.1725 - reconstruction_loss: 58.5936 - val_kl_loss: 1.6752 - val_loss: 26.8394 - val_reconstruction_loss: 25.1643
Epoch 78/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/st

Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - kl_loss: 0.1108 - loss: 65.6374 - reconstruction_loss: 65.5265 - val_kl_loss: 0.1791 - val_loss: 27.8509 - val_reconstruction_loss: 27.6719
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - kl_loss: 0.2094 - loss: 65.0999 - reconstruction_loss: 64.8905 - val_kl_loss: 0.2249 - val_loss: 27.8070 - val_reconstruction_loss: 27.5821
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - kl_loss: 0.2521 - loss: 65.0641 - reconstruction_loss: 64.8120 - val_kl_loss: 0.2565 - val_loss: 27.4291 - val_reconstruction_loss: 27.1726
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - kl_loss: 0.2957 - loss: 64.7981 - reconstruction_loss: 64.5024 - val_kl_loss: 0.3524 - val_loss: 27.6092 - val_reconstruction_loss: 27.2568
Epoch 13/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - kl_loss: 0.3851 - loss: 64.5234 - reconstruction_loss: 64.1383 - val_kl_loss: 0.4536 - val_loss: 27.6064 - val_reconstruction_loss: 27.1528
Epoch 14/100
4/4 ━━━━━━━━━━━━━━━━━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.9997 - loss: 59.5688 - reconstruction_loss: 57.5691 - val_kl_loss: 1.8856 - val_loss: 26.7576 - val_reconstruction_loss: 24.8720
Epoch 46/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.9988 - loss: 59.9072 - reconstruction_loss: 57.9085 - val_kl_loss: 1.9087 - val_loss: 26.7534 - val_reconstruction_loss: 24.8447
Epoch 47/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - kl_loss: 2.0466 - loss: 59.8519 - reconstruction_loss: 57.8053 - val_kl_loss: 1.9549 - val_loss: 26.8550 - val_reconstruction_loss: 24.9001
Epoch 48/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 2.0253 - loss: 59.5284 - reconstruction_loss: 57.5031 - val_kl_loss: 1.9325 - val_loss: 26.6524 - val_reconstruction_loss: 24.7199
Epoch 49/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 2.0652 - loss: 59.6994 - reconstruction_loss: 57.6342 - val_kl_loss: 2.0672 - val_loss: 26.8867 - val_reconstruction_loss: 24.8195
Epoch 50/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/st

...

Epoch 26/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.1099 - loss: 64.3471 - reconstruction_loss: 64.2372 - val_kl_loss: 0.0940 - val_loss: 27.2745 - val_reconstruction_loss: 27.1805
Epoch 27/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 0.1281 - loss: 64.7684 - reconstruction_loss: 64.6402 - val_kl_loss: 0.1514 - val_loss: 27.4838 - val_reconstruction_loss: 27.3325
Epoch 28/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - kl_loss: 0.1412 - loss: 64.5894 - reconstruction_loss: 64.4482 - val_kl_loss: 0.1443 - val_loss: 27.2426 - val_reconstruction_loss: 27.0983
Epoch 29/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.1762 - loss: 64.3597 - reconstruction_loss: 64.1835 - val_kl_loss: 0.1987 - val_loss: 27.5494 - val_reconstruction_loss: 27.3507
Epoch 30/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - kl_loss: 0.1777 - loss: 64.3814 - reconstruction_loss: 64.2037 - val_kl_loss: 0.1727 - val_loss: 27.2817 - val_reconstruction_loss: 27.1090
Epoch 31/100
4/4 ━━━━━━━━━━━━━━━━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.9832 - loss: 62.9139 - reconstruction_loss: 61.9307 - val_kl_loss: 1.1198 - val_loss: 27.1761 - val_reconstruction_loss: 26.0563
Epoch 63/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.1750 - loss: 62.4869 - reconstruction_loss: 61.3119 - val_kl_loss: 1.0388 - val_loss: 27.4301 - val_reconstruction_loss: 26.3914
Epoch 64/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.0265 - loss: 62.7158 - reconstruction_loss: 61.6893 - val_kl_loss: 0.9720 - val_loss: 26.8526 - val_reconstruction_loss: 25.8806
Epoch 65/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.0973 - loss: 62.8959 - reconstruction_loss: 61.7986 - val_kl_loss: 1.3311 - val_loss: 27.5699 - val_reconstruction_loss: 26.2388
Epoch 66/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.3162 - loss: 62.6366 - reconstruction_loss: 61.3205 - val_kl_loss: 1.0574 - val_loss: 27.0362 - val_reconstruction_loss: 25.9788
Epoch 67/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/st

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.4764 - loss: 61.4918 - reconstruction_loss: 60.0154 - val_kl_loss: 1.4193 - val_loss: 27.2343 - val_reconstruction_loss: 25.8150
Epoch 99/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 1.4246 - loss: 61.8224 - reconstruction_loss: 60.3978 - val_kl_loss: 1.3851 - val_loss: 27.2019 - val_reconstruction_loss: 25.8168
Epoch 100/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.4582 - loss: 62.0095 - reconstruction_loss: 60.5513 - val_kl_loss: 1.7014 - val_loss: 27.6452 - val_reconstruction_loss: 25.9438
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - kl_loss: 0.3288 - loss: 83.8283 - reconstruction_loss: 83.4995 - val_kl_loss: 0.0651 - val_loss: 32.3589 - val_reconstruction_loss: 32.2938
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.1336 - loss: 74.7009 - reconstruction_loss: 74.5673 - val_kl_loss: 0.4640 - val_loss: 29.6386 - val_reconstruction_loss: 29.1746
Epoch 3/100

Epoch 34/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 0.4833 - loss: 64.7027 - reconstruction_loss: 64.2194 - val_kl_loss: 0.4870 - val_loss: 27.4182 - val_reconstruction_loss: 26.9311
Epoch 35/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 0.5046 - loss: 64.0660 - reconstruction_loss: 63.5614 - val_kl_loss: 0.4588 - val_loss: 27.3994 - val_reconstruction_loss: 26.9406
Epoch 36/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - kl_loss: 0.4885 - loss: 63.9087 - reconstruction_loss: 63.4202 - val_kl_loss: 0.4631 - val_loss: 27.3767 - val_reconstruction_loss: 26.9136
Epoch 37/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.4973 - loss: 63.7337 - reconstruction_loss: 63.2365 - val_kl_loss: 0.5212 - val_loss: 27.3946 - val_reconstruction_loss: 26.8734
Epoch 38/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 0.5343 - loss: 64.0960 - reconstruction_loss: 63.5617 - val_kl_loss: 0.5636 - val_loss: 27.5017 - val_reconstruction_loss: 26.9382
Epoch 39/100
4/4 ━━━━━━━━━━━━━━━━━━

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.1219 - loss: 62.3246 - reconstruction_loss: 61.2026 - val_kl_loss: 1.1834 - val_loss: 27.1579 - val_reconstruction_loss: 25.9745
Epoch 71/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - kl_loss: 1.3608 - loss: 62.2410 - reconstruction_loss: 60.8802 - val_kl_loss: 1.3030 - val_loss: 27.1427 - val_reconstruction_loss: 25.8398
Epoch 72/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.3412 - loss: 62.1211 - reconstruction_loss: 60.7799 - val_kl_loss: 1.1736 - val_loss: 27.2503 - val_reconstruction_loss: 26.0767
Epoch 73/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.2433 - loss: 62.2526 - reconstruction_loss: 61.0093 - val_kl_loss: 1.0374 - val_loss: 26.9508 - val_reconstruction_loss: 25.9134
Epoch 74/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - kl_loss: 1.1636 - loss: 62.6741 - reconstruction_loss: 61.5106 - val_kl_loss: 1.1273 - val_loss: 27.1060 - val_reconstruction_loss: 25.9787
Epoch 75/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/st

In [ ]:
path = "dz_vae/vae_" + str(4) + ".csv" 

In [ ]:
path

In [ ]:
history = vae.fit(
    student_trn,
    epochs=100,
    batch_size = 128,
    validation_data=student_val,
    shuffle=True
)

In [ ]:
student_vae = vaes[1].predict(student_tst)[2]

In [ ]:
student_vae

In [ ]:
student_vae = pd.DataFrame(student_scaler.inverse_transform(student_vae), 
                             columns = student_cols)
enc_cols = student_enc.inverse_transform(student_vae[student_enc.get_feature_names_out()])
student_vae = student_vae.drop(student_enc.get_feature_names_out(), axis = 1)
student_vae[student_enc.feature_names_in_] = enc_cols
student_vae.to_csv("dz_vae/vae_dz1.csv", index=False)
student_vae.head()

In [ ]:
student_tst

In [ ]:
smth = pd.read_csv("ae/ae_student.csv")

In [ ]:
smth

In [ ]:
smth_2 = pd.read_csv("student_tst.csv")
smth_2

## DZ = 2

In [ ]:
encoder = VEncoder(input_dim, 2)
decoder = VDecoder(2, input_dim)

vae = VAE(encoder, decoder)

optimizer = tf.keras.optimizers.AdamW(5e-3)
vae.compile(optimizer=optimizer)

In [ ]:
history = vae.fit(
    student_trn,
    epochs=50,
    batch_size = 128,
    validation_data=student_val,
    shuffle=True
)

In [ ]:
student_vae = vae.predict(student_tst)

In [ ]:
student_vae[2]

In [ ]:
student_vae

In [ ]:
student_vae = pd.DataFrame(student_scaler.inverse_transform(student_vae), 
                             columns = student_cols)
enc_cols = student_enc.inverse_transform(student_vae[student_enc.get_feature_names_out()])
student_vae = student_vae.drop(student_enc.get_feature_names_out(), axis = 1)
student_vae[student_enc.feature_names_in_] = enc_cols
student_vae.to_csv("dz_vae/vae_dz2.csv", index=False)
student_vae.head()

In [ ]:
student_vae

In [ ]:
student_tst